# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Wednesday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**


# Question 1 (40 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]

(3) Collect all the reviews of the top 1000 most popular software from G2 or Capterra.

(4) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(5) Collect all the information of the 904 narrators in the Densho Digital Repository.


In [16]:
import requests
from bs4 import BeautifulSoup
import csv

def fetch_imdb_reviews(movie_links):
    aggregated_reviews = []
    for movie_link in movie_links:
        page_counter = 0
        while len(aggregated_reviews) < 1000:
            constructed_url = f"{movie_link[0]}?sort=userRating&dir=desc&ratingFilter=0&start={page_counter * 25}"
            response = requests.get(constructed_url)
            if response.ok:
                soup = BeautifulSoup(response.text, 'html.parser')
                section_reviews = soup.select('.lister-list .review-container')
                if not section_reviews:
                    print(f"No more reviews found for URL: {constructed_url}")
                    break
                for review in section_reviews:
                    text_section = review.select_one('.text.show-more__control')
                    review_text = text_section.text if text_section else "No review text."
                    rating_section = review.select_one('.rating-other-user-rating span')
                    review_rating = rating_section.text if rating_section else "No rating."
                    aggregated_reviews.append([review_text, review_rating])
                    if len(aggregated_reviews) == 1000:
                        break
                page_counter += 1
            else:
                print(f"Error retrieving reviews from {constructed_url}")
                break
        if len(aggregated_reviews) >= 1000:
            break
    return aggregated_reviews

def saved_reviews(reviews, file_name):
    headers = ['Review Text', 'Rating']
    with open(file_name, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        for review in reviews:
            writer.writerow(review)

def main():
    movie_url = [
        ('https://www.imdb.com/title/tt1462764/reviews',)
    ]
    reviews = fetch_imdb_reviews(movie_url)
    saved_reviews(reviews, 'imdb_reviews_output.csv')
    print(f"Saved {len(reviews)} reviews to 'imdb_reviews_output.csv'.")

if __name__ == '__main__':
    main()


Saved 1000 reviews to 'imdb_reviews_output.csv'.


# Question 2 (30 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [17]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

df = pd.read_csv('imdb_reviews_output.csv')

def clean_text(text):
    text = re.sub(r'[^A-Za-z\s]', '', text)
    text = re.sub(r'\d+', '', text)
    words = nltk.word_tokenize(text)
    words = [word for word in words if word.lower() not in stopwords.words('english')]
    words = [word.lower() for word in words]
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    cleaned_data = ' '.join(words)
    return cleaned_data

df['Cleaned Review Text'] = df['Review Text'].apply(clean_text)

df.to_csv('imdb_reviews_cleaned.csv', index=False)

print("Finished cleaning and saved the cleaned data to 'imdb_reviews_cleaned.csv'.")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Finished cleaning and saved the cleaned data to 'imdb_reviews_cleaned.csv'.


# Question 3 (30 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [18]:
import pandas as pd
import spacy
from collections import Counter

nlp = spacy.load("en_core_web_sm")

df = pd.read_csv('imdb_reviews_cleaned.csv')

def parts_of_speech_tagging(text):
    doc = nlp(text)
    parts_of_speech_count = Counter(token.pos_ for token in doc)
    return parts_of_speech_count

def constituency_dependency_parsing(text):
    doc = nlp(text)
    for token in doc:
        print(f"{token.text} -> {token.dep_} -> {token.head.text}")

def named_entity_recognition(text):
    doc = nlp(text)
    entities = [ent.label_ for ent in doc.ents]
    return Counter(entities)


example_text = df['Cleaned Review Text'].iloc[0]
print(f"Example Text: {example_text}\n")

parts_of_speech_count = parts_of_speech_tagging(example_text)
print("POS Tag Counts:", parts_of_speech_count)

print("\nDependency Parsing (first sentence):")
constituency_dependency_parsing(example_text.split('.')[0])

entity_counts = named_entity_recognition(' '.join(df['Cleaned Review Text']))
print("\nNamed Entity Counts:", entity_counts)



Example Text: movi pan critic loss element weve come expect indi indcred breathtak action sequenc car chase hors full gallop mysteri artifact memor bad guy question good guy andharrison ford year old still fantast job action hero trhow nowexpect work luca spielberg ahv lavishli produc beauti shot piec cinemat entertain nobl prizewin literatur win award academi exactli suppos give u hour highli engag entertain valu money top bring harrison ford tenur indi satisfi conclus hold true origin portray main charact

POS Tag Counts: Counter({'PROPN': 33, 'NOUN': 26, 'VERB': 9, 'ADJ': 8, 'PRON': 1, 'AUX': 1, 'ADV': 1})

Dependency Parsing (first sentence):
movi -> compound -> element
pan -> compound -> element
critic -> compound -> element
loss -> compound -> element
element -> nsubj -> expect
we -> nsubj -> come
ve -> aux -> come
come -> relcl -> element
expect -> ROOT -> expect
indi -> nsubj -> indcred
indcred -> ccomp -> expect
breathtak -> nmod -> hors
action -> compound -> sequenc
sequenc -

# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

In [19]:
"Fetching 1000 reviews from imdb is really challenging, using chatgpt and understanding the logic behind I kind of learn debugging as well."

'Fetching 1000 reviews from imdb is really challenging, using chatgpt and understanding the logic behind I kind of learn debugging as well.'